In [2]:
from __future__ import print_function
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from pandas import DataFrame
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import math
import seaborn as sns
from sklearn import datasets
import random
from collections import Counter

In [3]:
군집 = pd.read_excel('./0818 군집분석용_2019년 동별 자료.xlsx', 
              sheet_name = 'data')

In [6]:
군집.columns

Index(['19년 인구', '남녀성비', '통', '면적', '1인 가구', '2인 가구', '3-4인_가구', '다가구(5인_이상)',
       '19년 미성년자 유동인구 비율(빅콘)', '19년 노인유동인구 비율(빅콘)',
       '19년 숙박업소 이용건수/ 전체이용건수 (빅콘)', '19년 요식업소 이용건수/전체이용건수 (빅콘)',
       '60세 이상 거주 비율', '미성년자 거주 비율', '면적 별', '면적별_공동주택', '면적별_병의원', '면적별_숙박업',
       '편의점_유무', '편의점_식사', '편의점_간식', '편의점_마실거리', '편의점_홈/리빙', '편의점_헬스/뷰티',
       '편의점_취미/여가', '편의점_사회활동', '편의점_임신/육아', '편의점_평균매출', '편의점_최대매출',
       '편의점_최소매출', '단독주택', '아파트', '연립주택', '다세대주택', '기타', '접근_자차_광역교통',
       '접근_자차_교육시설', '접근_자차_의료시설', '접근_자차_판매시설', '인구당_전산업', '인구당_제조업',
       '인구당_도소매업', '인구당_운수_창고업', '인구당_숙박_음식점', '인구당_금융보험업', '인구당_부동산업',
       '인구당_교육서비스업', '인구당_사회복지업', '인구당_레저예술', '전체 인구 당(%)', '평균연령', '중위연령',
       '인구밀도'],
      dtype='object')

In [4]:
군집.set_index('행정동', inplace=True)

In [5]:
target = 군집.columns
pd.set_option('display.max_columns', None)


# 군집분석에서 이정도면 정확하다고 할 만한 socre 기준치 값, 미리 임의로 정해놓기(여기서는 일단 500으로 설정)
temp_std_week = -300
temp_std_strong = -50


# 몇번 돌려볼 것인가(일단 30으로 가정)
반복 = 30

# 반드시 모여있어야 하는 친구들 1번째, 일단 임의로 3개 설정
must_have_1 = ['성내1동','성내2동','범어1동']

# 혹시 더 있으면
must_have_2 = []
must_have_3 = []

In [ ]:
# 예쁘게 모으는 거에 성공하면, 성공적인 칼럼들이 무엇인지 저장 할 칼럼
features = []
temp_list = []


# 마지막에 정리 할 때 쓸거야
얼마나_나왔니 = pd.DataFrame(columns=군집.columns)
x = 1


for i in range(0,반복):
    temp_list = []
    # 총 53개의 컬럼 중에 몇개나 뽑을지 무작위로 정함(randrange) = 몇개 뽑을지 picks에 저장
    picks = random.randrange(1,53)
    
    # picks에 5개가 나오면 5개, 10개가 나오면 10개, 기존의 데이터 칼럼 리스트에서 뽑는다.
    sampling = random.choices(target, k = picks)
    sampling = set(sampling)
    sampling = list(sampling)
    
    # 예를들어 인구밀도, 성비를 컬럼에서 뽑았다면 이에 해당하는 컬럼들만 모아서 데이터 프레임을 만든다.
    target_data = 군집[sampling]

    
    ################################ 군집분석 모델링 ############################################

    
    # 정규화를 어떻게 할건지, 무슨 군집분석 모델이 넣을 건지 정해지면 넣겠습니다.
    # 일단 임시로 가장 간단한 모델에 넣어서 돌아가는지 확인한다.
    data_standadized = StandardScaler().fit_transform(target_data)
    model1 = KMeans(n_clusters=3, init='random', n_init=1000, max_iter=1000).fit(data_standadized)
    cs0,cs1,cs2 = model1.cluster_centers_
    print(i+1, '번째 작업 중...')
    
    
    ################################# 군집분석 모델링 ############################################
    
    
    # 우리가 임의로 설정한 답안들이 잘 모여있는지 확인한다.
    label_1 = model1.predict(target_data.loc[must_have_1])
    # 전부 1,1,1 or 2,2,2 이렇게 나오면 set 했을 때 길이가 1로 나올 것(동일한 세개 이므로)
    if len(set(label_1)) == 1:

        
        if picks > 3:
            if model1.score(data_standadized) > temp_std_week:
                print('유효 Features(특징) 발견!!')
                # 각각의 점수를 측정한다.
                print('점수 :', model1.score(data_standadized))
                print('특징의 개수 :', picks)
                print('사용한 특징 :', sampling)
                for i in range (0, len(sampling)):
                    features.append(sampling[i])

                for i in range (0, len(target)):
                    if target[i] in sampling:
                        temp_list.append(1)
                    else:
                        temp_list.append(0)
                얼마나_나왔니.loc[x] = temp_list
                x += 1        
                        

            else:
                print('분류 실패!')
                
                                        
        else:
            if model1.score(data_standadized) > temp_std_strong:
                print('유효 Features(특징) 발견!!')
                # 각각의 점수를 측정한다.
                print('점수 :', model1.score(data_standadized))
                print('특징의 개수 :', picks)
                print('사용한 특징 :', sampling)
                for i in range (0, len(sampling)):
                    features.append(sampling[i])
                    
                for i in range (0, len(target)):
                    if target[i] in sampling:
                        temp_list.append(1)
                    else:
                        temp_list.append(0)
                얼마나_나왔니.loc[x] = temp_list
                x += 1
                    

            else:
                print('분류 실패!')
    else:
        print('분류 실패!')
        

result = Counter(features)
print('-------------------------------------------(성공했을 때)컬럼들이 몇번이나 뽑혔나-----------------------------------------------')
for key in result:
    print (key,":", result[key], '번 등장')

1 번째 작업 중...
분류 실패!
2 번째 작업 중...
분류 실패!
3 번째 작업 중...
분류 실패!
4 번째 작업 중...
분류 실패!
5 번째 작업 중...
분류 실패!
6 번째 작업 중...
분류 실패!
7 번째 작업 중...
유효 Features(특징) 발견!!
점수 : -265.41171695491397
특징의 개수 : 6
사용한 특징 : ['60세 이상 거주 비율', '면적 별', '접근_자차_광역교통', '통', '편의점_홈/리빙', '편의점_사회활동']
8 번째 작업 중...
분류 실패!
9 번째 작업 중...
유효 Features(특징) 발견!!
점수 : -155.70161557502186
특징의 개수 : 4
사용한 특징 : ['다가구(5인_이상)', '편의점_식사', '인구당_부동산업', '연립주택']
10 번째 작업 중...
분류 실패!
11 번째 작업 중...
분류 실패!
12 번째 작업 중...
분류 실패!
13 번째 작업 중...
분류 실패!
14 번째 작업 중...
분류 실패!
15 번째 작업 중...
분류 실패!
16 번째 작업 중...
분류 실패!
17 번째 작업 중...
분류 실패!
18 번째 작업 중...
분류 실패!
19 번째 작업 중...
분류 실패!
20 번째 작업 중...
분류 실패!
21 번째 작업 중...
분류 실패!
22 번째 작업 중...
분류 실패!
23 번째 작업 중...
분류 실패!
24 번째 작업 중...
분류 실패!
25 번째 작업 중...
분류 실패!
26 번째 작업 중...
분류 실패!
27 번째 작업 중...
분류 실패!
28 번째 작업 중...
분류 실패!
29 번째 작업 중...
분류 실패!
30 번째 작업 중...
유효 Features(특징) 발견!!
점수 : -3.4290392568154635
특징의 개수 : 1
사용한 특징 : ['인구당_레저예술']
31 번째 작업 중...
분류 실패!
32 번째 작업 중...
분류 실패!
33 번째 작업 중...
분류 실패!
34 번째 작업 중...

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_fast._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt: 


38 번째 작업 중...
유효 Features(특징) 발견!!
점수 : -293.18700441128715
특징의 개수 : 7
사용한 특징 : ['편의점_최대매출', '인구당_전산업', '접근_자차_광역교통', '통', '편의점_마실거리', '전체 인구 당(%)', '면적별_공동주택']
39 번째 작업 중...
유효 Features(특징) 발견!!
점수 : -248.47837292297334
특징의 개수 : 8
사용한 특징 : ['인구당_전산업', '아파트', '접근_자차_판매시설', '접근_자차_광역교통', '19년 노인유동인구 비율(빅콘)', '전체 인구 당(%)']
40 번째 작업 중...
분류 실패!
41 번째 작업 중...
분류 실패!
42 번째 작업 중...
분류 실패!
43 번째 작업 중...
분류 실패!
44 번째 작업 중...
분류 실패!


In [16]:
얼마나_나왔니

,19년 인구,남녀성비,통,면적,1인 가구,2인 가구,3-4인_가구,다가구(5인_이상),19년 미성년자 유동인구 비율(빅콘),19년 노인유동인구 비율(빅콘),19년 숙박업소 이용건수/ 전체이용건수 (빅콘),19년 요식업소 이용건수/전체이용건수 (빅콘),60세 이상 거주 비율,미성년자 거주 비율,면적 별,면적별_공동주택,면적별_병의원,면적별_숙박업,편의점_유무,편의점_식사,편의점_간식,편의점_마실거리,편의점_홈/리빙,편의점_헬스/뷰티,편의점_취미/여가,편의점_사회활동,편의점_임신/육아,편의점_평균매출,편의점_최대매출,편의점_최소매출,단독주택,아파트,연립주택,다세대주택,기타,접근_자차_광역교통,접근_자차_교육시설,접근_자차_의료시설,접근_자차_판매시설,인구당_전산업,인구당_제조업,인구당_도소매업,인구당_운수_창고업,인구당_숙박_음식점,인구당_금융보험업,인구당_부동산업,인구당_교육서비스업,인구당_사회복지업,인구당_레저예술,전체 인구 당(%),평균연령,중위연령,인구밀도
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
10,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
